In [0]:
df_sales = spark.read.csv("dbfs:/FileStore/tables/sales_transaction06_12_2020_10_20_33.csv",header=True)

In [0]:
display(df_sales.limit(10))

PROD_ID,CUST_ID,TIME_ID,CHANNEL_ID,PROMO_ID,QUANTITY_SOLD,AMOUNT_SOLD
13,987,10-01-98,3,999,1,1232.16
13,1660,10-01-98,3,999,1,1232.16
13,1762,10-01-98,3,999,1,1232.16
13,1843,10-01-98,3,999,1,1232.16
13,1948,10-01-98,3,999,1,1232.16
13,2273,10-01-98,3,999,1,1232.16
13,2380,10-01-98,3,999,1,1232.16
13,2683,10-01-98,3,999,1,1232.16
13,2865,10-01-98,3,999,1,1232.16
13,4663,10-01-98,3,999,1,1232.16


In [0]:
from pyspark.sql.functions import concat,col,crc32,row_number
from pyspark.sql.window import Window
df_sales = df_sales.withColumn("id",concat(col("PROD_ID"),col("CUST_ID"),col("TIME_ID"),col("CHANNEL_ID")))
df_sales = df_sales.withColumn("crc32key",crc32(col("id")))
df_sales = df_sales.withColumn("duplicates",row_number().over(Window.partitionBy("crc32key").orderBy("crc32key")))
df_sales.createOrReplaceTempView("sales_v")

In [0]:
%sql
select * from sales_v where crc32key in (select crc32key from sales_v where duplicates>1)

PROD_ID,CUST_ID,TIME_ID,CHANNEL_ID,PROMO_ID,QUANTITY_SOLD,AMOUNT_SOLD,id,crc32key,duplicates
27,4607,20-10-98,2,999,1,49.27,27460720-10-982,11923631,1
34,1428,28-07-01,4,999,1,40.39,34142828-07-014,11923631,2
113,4044,01-08-00,2,999,1,24.58,113404401-08-002,114812311,1
139,453,18-10-01,4,999,1,20.22,13945318-10-014,114812311,2
118,9723,01-01-00,3,999,1,8.86,118972301-01-003,170251350,1
30,7784,03-09-00,2,999,1,9.35,30778403-09-002,170251350,2
131,13159,24-07-98,2,999,1,26.02,1311315924-07-982,183615622,1
125,28323,18-12-98,2,999,1,16.61,1252832318-12-982,183615622,2
125,5723,29-01-98,3,999,1,16.63,125572329-01-983,201585979,1
117,4273,27-04-99,3,999,1,11.11,117427327-04-993,201585979,2


In [0]:
%sql
select crc32("118972301-01-003")
union all
select crc32("30778403-09-002")

crc32(118972301-01-003)
170251350
170251350
